In [ ]:
=================================================
We have this function as you can see it accept a single PDF path and then
convert it into collection of images per page and then
- read pdf text for all pages and save it into sub_pdf_page_text list
and also list of all conveted images into sub_images_path LIST

and then in a for loop :
for count, (image_path, pdf_text) in enumerate(zip(sub_images_path, sub_pdf_page_text)):

  as you can see call OCR function "run_image_text_extraction"
  per page
  and then we update
  inputs_payload per page
  and then send it per page to
  ContractcleanPDFCrew().crew().kickoff(inputs=inputs_payload)

  as a result of this
  we have some text files a folder and then we merge it
  merge_txt_files(file_folder)

  I wpuld like to accelarate the procedure in for loop with using ray or multu threading or multi processing

  ==============================================


from pathlib import Path
import os
from PyPDF2 import PdfReader
from slugify import slugify

def kickoff_ray(file_path, output_folder='output/images'):
    inputs_payload = {'pdf_path': file_path}

    # Convert PDF to JPG
    sub_images_list_with_metaData = pdf2jpg.convert_pdf2jpg(
        file_path, output_folder, pages="ALL"
    )

    # Extract text from PDF
    reader_pdf = PdfReader(file_path)
    sub_pdf_page_text = [page.extract_text() for page in reader_pdf.pages]

    # Extract image paths
    sub_images_path = sub_images_list_with_metaData[0]['output_jpgfiles']
    sub_images_list_count = len(sub_images_path)
    folder_to_delete = os.path.dirname(sub_images_path[0])

    print("folder_to_delete\n", folder_to_delete)

    # Check if the image count matches PDF page count
    if len(sub_images_path) != len(sub_pdf_page_text):
        raise ValueError("The image paths and PDF texts lists must have the same length !!!")
    else:
        print("The same Length for both OCR Sub-Images and PDF Sub-Pages :\n")

    # Loop through images and PDF text
    for count, (image_path, pdf_text) in enumerate(zip(sub_images_path, sub_pdf_page_text)):
        print(f"\nProcessing Pair {count+1}/{len(sub_images_path)}")
        extracted_text_ocr = run_image_text_extraction(image_path)
        print(f"Image : {image_path}")

        # Extract base file name and create folder name
        file_name = os.path.splitext(os.path.basename(inputs_payload['pdf_path']))[0]
        file_folder = slugify_filename(file_name)

        print(f"Filename is {file_name}")
        print(f"File Folder is {file_folder}")

        # Create the directory for JSON parts
        print(f"Creating output directory: {file_folder}")
        Path(f"output/json/{file_folder}").mkdir(parents=True, exist_ok=True)

        print(f"Text Length : {len(pdf_text)} characters\n")

        # Update the inputs payload with extracted data
        inputs_payload.update({
            'image_path': image_path,
            'file_name': file_name,
            'file_folder': file_folder,
            'page_number': count + 1,
            'pdf_text': pdf_text,
            'ocr_text': extracted_text_ocr
        })

        # Kick off processing crew
        print(f"\nInput PAYLOAD Keys : Kicking off inputs : {inputs_payload.keys()}")
        print("======================")
        ContractcleanPDFCrew().crew().kickoff(inputs=inputs_payload)

    # Merge all text files for the contract
    print("Merging pages Per input Contract file; \n")
    merge_txt_files(file_folder)
    clean_text_path=os.path.join(.........)
    input_payload_parsing ={'clean_text_path': clean_text_path}
    contractanalysis crew().crew().kickoff(inputs- input_payload_parsing)



In [ ]:
import ray
from pathlib import Path
import os
from PyPDF2 import PdfReader
from slugify import slugify

ray.init(ignore_reinit_error=True)

# Your OCR + crew function wrapped in a Ray task
@ray.remote
def process_page_remote(inputs_payload_base, image_path, pdf_text, count):
    from your_module import run_image_text_extraction, ContractcleanPDFCrew  # import inside Ray task
    from your_module import slugify_filename  # helper if not already global

    extracted_text_ocr = run_image_text_extraction(image_path)

    file_name = os.path.splitext(os.path.basename(inputs_payload_base['pdf_path']))[0]
    file_folder = slugify_filename(file_name)
    Path(f"output/json/{file_folder}").mkdir(parents=True, exist_ok=True)

    # Prepare per-page payload
    inputs_payload = inputs_payload_base.copy()
    inputs_payload.update({
        'image_path': image_path,
        'file_name': file_name,
        'file_folder': file_folder,
        'page_number': count + 1,
        'pdf_text': pdf_text,
        'ocr_text': extracted_text_ocr
    })

    ContractcleanPDFCrew().crew().kickoff(inputs=inputs_payload)
    return file_folder  # return file_folder for merging step later


In [ ]:
def kickoff_ray(file_path, output_folder='output/images'):
    inputs_payload = {'pdf_path': file_path}

    sub_images_list_with_metaData = pdf2jpg.convert_pdf2jpg(file_path, output_folder, pages="ALL")
    reader_pdf = PdfReader(file_path)
    sub_pdf_page_text = [page.extract_text() for page in reader_pdf.pages]
    sub_images_path = sub_images_list_with_metaData[0]['output_jpgfiles']

    if len(sub_images_path) != len(sub_pdf_page_text):
        raise ValueError("Image and PDF page counts mismatch!")

    # Launch Ray tasks for each page
    futures = [
        process_page_remote.remote(inputs_payload, image_path, pdf_text, count)
        for count, (image_path, pdf_text) in enumerate(zip(sub_images_path, sub_pdf_page_text))
    ]

    # Wait for all Ray tasks to complete and get file folder name
    file_folder_list = ray.get(futures)
    file_folder = file_folder_list[0]  # all should be the same

    # Merge results after all parallel tasks
    print("Merging pages per input contract file;")
    merge_txt_files(file_folder)

    clean_text_path = os.path.join("output/json", file_folder, "merged.txt")
    input_payload_parsing = {'clean_text_path': clean_text_path}
    contractanalysiscrew().crew().kickoff(inputs=input_payload_parsing)


In [ ]:
import os
from pathlib import Path
from PyPDF2 import PdfReader
from slugify import slugify
import ray

# Initialize Ray
ray.init(ignore_reinit_error=True)

# === Dummy Replacements: Replace with your actual implementations === #
def pdf2jpg_convert_dummy(file_path, output_folder, pages="ALL"):
    # Simulate conversion, return structure similar to pdf2jpg
    return [{
        'output_jpgfiles': [f"{output_folder}/page_{i}.jpg" for i in range(3)]
    }]

def run_image_text_extraction(image_path):
    # Simulated OCR function
    return f"Extracted OCR text from {image_path}"

class ContractcleanPDFCrew:
    def crew(self):
        return self
    def kickoff(self, inputs):
        print(f"[Crew] Kicked off with inputs: {list(inputs.keys())}")

class contractanalysiscrew:
    def crew(self):
        return self
    def kickoff(self, inputs):
        print(f"[Analysis] Analyzing: {inputs['clean_text_path']}")

def merge_txt_files(folder):
    # Simulated merge function
    print(f"[Merge] Merging text files in folder: {folder}")
# ===================================================================== #

# Helper to slugify filename
def slugify_filename(name):
    return slugify(name)

# Ray remote task to process a single page
@ray.remote
def process_page_remote(inputs_payload_base, image_path, pdf_text, count):
    extracted_text_ocr = run_image_text_extraction(image_path)

    file_name = os.path.splitext(os.path.basename(inputs_payload_base['pdf_path']))[0]
    file_folder = slugify_filename(file_name)
    Path(f"output/json/{file_folder}").mkdir(parents=True, exist_ok=True)

    inputs_payload = inputs_payload_base.copy()
    inputs_payload.update({
        'image_path': image_path,
        'file_name': file_name,
        'file_folder': file_folder,
        'page_number': count + 1,
        'pdf_text': pdf_text,
        'ocr_text': extracted_text_ocr
    })

    ContractcleanPDFCrew().crew().kickoff(inputs=inputs_payload)
    return file_folder

# Main function to kick off processing
def kickoff_ray(file_path, output_folder='output/images'):
    inputs_payload = {'pdf_path': file_path}

    # === Convert PDF to Images ===
    sub_images_list_with_metaData = pdf2jpg_convert_dummy(file_path, output_folder)
    sub_images_path = sub_images_list_with_metaData[0]['output_jpgfiles']

    # === Extract PDF Text ===
    reader_pdf = PdfReader(file_path)
    sub_pdf_page_text = [page.extract_text() or "" for page in reader_pdf.pages]

    if len(sub_images_path) != len(sub_pdf_page_text):
        raise ValueError("Mismatch in image and text lengths")

    # === Launch Ray tasks ===
    futures = [
        process_page_remote.remote(inputs_payload, img, txt, idx)
        for idx, (img, txt) in enumerate(zip(sub_images_path, sub_pdf_page_text))
    ]

    file_folders = ray.get(futures)
    file_folder = file_folders[0]  # all pages share the same folder

    # === Post-processing ===
    merge_txt_files(file_folder)
    clean_text_path = os.path.join("output/json", file_folder, "merged.txt")
    contractanalysiscrew().crew().kickoff(inputs={'clean_text_path': clean_text_path})

# === Run example ===
if __name__ == "__main__":
    test_pdf_path = "sample_contract.pdf"
    kickoff_ray(test_pdf_path)
